### Property Recommendation System

In [1]:
import numpy as np
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('appraisals_dataset.json', 'r') as f:
    appraisals_dataset = pd.read_json(f)
print("Data shape:", appraisals_dataset.shape)

df = pd.json_normalize(appraisals_dataset['appraisals'].tolist())
df.to_csv('appraisals.csv', index=False)
df = pd.read_csv('appraisals.csv')

Data shape: (88, 1)


In [3]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)

Training set shape: (70, 38)
Testing set shape: (18, 38)


### Data Pre-processing

In [4]:
# X_train
import ast

properties = train_df[['orderID', 'properties']]

records = []
for _, row in properties.iterrows():
    orderID = row['orderID']
    prop_list = row['properties']
    prop_list = ast.literal_eval(prop_list)
    if isinstance(prop_list, str):
        try:
            prop_list = json.loads(prop_list.replace("'", '"'))
        except Exception as e:
            continue
    for prop in prop_list:
        prop_record = {
            "orderID": orderID,
            "id":prop.get("id", None),
            "address": prop.get("address", None),
            "bedrooms": prop.get("bedrooms", None),
            "gla": prop.get("gla", None),
            "city": prop.get("city", None),
            "province": prop.get("province", None),
            "postal_code": prop.get("postal_code", None),
            "property_sub_type": prop.get("property_sub_type", None),
            "structure_type": prop.get("structure_type", None),
            "style": prop.get("style", None),
            "levels": prop.get("levels", None),
            "room_count": prop.get("room_count", None),
            "full_baths": prop.get("full_baths", None),
            "half_baths": prop.get("half_baths", None),
            "lot_size_sf": prop.get("lot_size_sf", None),
            "year_built": prop.get("year_built", None),
            "roof": prop.get("roof", None),
            "basement": prop.get("basement", None),
            "cooling": prop.get("cooling", None),
            "heating": prop.get("heating", None),
        }
        records.append(prop_record)

properties_df = pd.DataFrame(records)
properties_df = properties_df.rename(columns={
    col: f"prop.{col}" for col in properties_df.columns if col != "orderID"
})
merged_train_df = train_df.merge(properties_df, on='orderID', how='left')
# X_train = merged_train_df.drop(columns=columns_to_drop_X)

In [5]:
# X_test
import ast

properties = test_df[['orderID', 'properties']]

records = []
for _, row in properties.iterrows():
    orderID = row['orderID']
    prop_list = row['properties']
    prop_list = ast.literal_eval(prop_list)
    if isinstance(prop_list, str):
        try:
            prop_list = json.loads(prop_list.replace("'", '"'))
        except Exception as e:
            continue
    for prop in prop_list:
        prop_record = {
            "orderID": orderID,
            "id":prop.get("id", None),
            "address": prop.get("address", None),
            "bedrooms": prop.get("bedrooms", None),
            "gla": prop.get("gla", None),
            "city": prop.get("city", None),
            "province": prop.get("province", None),
            "postal_code": prop.get("postal_code", None),
            "property_sub_type": prop.get("property_sub_type", None),
            "structure_type": prop.get("structure_type", None),
            "style": prop.get("style", None),
            "levels": prop.get("levels", None),
            "room_count": prop.get("room_count", None),
            "full_baths": prop.get("full_baths", None),
            "half_baths": prop.get("half_baths", None),
            "lot_size_sf": prop.get("lot_size_sf", None),
            "year_built": prop.get("year_built", None),
            "roof": prop.get("roof", None),
            "basement": prop.get("basement", None),
            "cooling": prop.get("cooling", None),
            "heating": prop.get("heating", None),
        }
        records.append(prop_record)

properties_df = pd.DataFrame(records)
properties_df = properties_df.rename(columns={
    col: f"prop.{col}" for col in properties_df.columns if col != "orderID"
})
merged_test_df = train_df.merge(properties_df, on='orderID', how='left')
# X_test = merged_test_df.drop(columns=columns_to_drop_X)

In [6]:
import re

import pandas as pd
import re

def standardize_and_split_address(address):
    
    if pd.isna(address):
        return pd.Series([None, None, None, None])
    
    addr = str(address).strip().replace('"', '').replace("'", "")
    
    # Extract postal code using Canadian postal code pattern
    postal_code_pattern = r'([A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d)'
    postal_code_match = re.search(postal_code_pattern, addr)
    postal_code = postal_code_match.group(1) if postal_code_match else None
    
    if postal_code:
        addr_wo_postal = addr.replace(postal_code, '').strip(', ')
    else:
        addr_wo_postal = addr
    
    # Check if address contains commas
    if ',' in addr_wo_postal:
        parts = [p.strip() for p in addr_wo_postal.split(',') if p.strip()]
        
        province = None
        city = None
        address_part = None
        
        if len(parts) >= 2:
            province_match = re.search(r'\b([A-Z]{2})\b', parts[-1])
            if province_match:
                province = province_match.group(1)
                city = parts[-2]
                address_part = ', '.join(parts[:-2])
            else:
                city = parts[-1]
                address_part = ', '.join(parts[:-1])
        elif len(parts) == 1:
            address_part = parts[0]
        else:
            address_part = addr_wo_postal
    else:
        # Space separated, try to parse from tokens
        tokens = addr_wo_postal.split()
        
        province = None
        city = None
        address_part = None
        
        # Assume last token is province if it is 2 uppercase letters
        if len(tokens) >= 2 and re.fullmatch(r'[A-Z]{2}', tokens[-1]):
            province = tokens[-1]
            if province == "ON":
                province = "Ontario"
            elif province == "BC":
                province = "British Columbia"
            elif province == "AB":
                province = "Alberta"
            elif province == "NS": 
                province = "Nova Scotia"
            elif province == "None":
                return np.nan
            city = tokens[-2]
            address_part = " ".join(tokens[:-2])
        elif len(tokens) >= 1:
            # fallback if no province token detected
            address_part = " ".join(tokens[:-1])
            city = tokens[-1]
        else:
            address_part = addr_wo_postal
    
    return pd.Series([address_part, city, province, postal_code])

# Apply to normalized_df
merged_train_df[['subject.address', 'subject.city', 'subject.province', 'subject.postal_code']] = merged_train_df['subject.address'].apply(standardize_and_split_address)
merged_test_df[['subject.address', 'subject.city', 'subject.province', 'subject.postal_code']] = merged_test_df['subject.address'].apply(standardize_and_split_address)

In [7]:
def parse_gla(value):
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value.replace('.','',1).isdigit():
        
        return float(value)
    value = value.replace('+/-','')
    match = re.search(r"([\d.]+)", value)
    if not match:
        return np.nan
    number = float(match[0]) 
    if any(x in value for x in ['sqm']):
        return number * 10.7639
    elif any(x in value for x in ['sqft', 'sf']):
        return float(number)
    else:
        return np.nan
    


merged_train_df['subject.gla_parsed'] = merged_train_df['subject.gla'].apply(parse_gla)
merged_test_df['subject.gla_parsed'] = merged_test_df['subject.gla'].apply(parse_gla)

In [8]:
merged_train_df['subject.gla_parsed']

0       1504.0
1       1504.0
2       1504.0
3       1504.0
4       1504.0
         ...  
8124       1.0
8125       1.0
8126       1.0
8127       1.0
8128       1.0
Name: subject.gla_parsed, Length: 8129, dtype: float64

In [9]:

def parse_year_built(value):
    if pd.isna(value):
        return np.nan
    
    value = str(value).strip().lower()
    if value == "new":
        return 2025
    if value.isdigit():
        return float(value)
    if value.replace('.','',1).isdigit():
        return float(value)
    match = re.search(r"([\d.]+)", value)
    if not match:
        return np.nan
    number = float(match[0]) 
    return number

merged_train_df['subject.year_built_parsed'] = merged_train_df['subject.year_built'].apply(parse_year_built)
merged_test_df['subject.year_built_parsed'] = merged_test_df['subject.year_built'].apply(parse_year_built)

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime

def clean_year_built(value):
    try:
        # Convert from scientific notation or string to float
        year = float(value)
        year = int(round(year))  # Convert to integer
        
        current_year = datetime.now().year
        
        # Handle likely invalid placeholder years (e.g., 40, 8, 13)
        if year < 100:  
            return np.nan

        # Handle likely valid years (e.g., 1900–current year)
        if 1800 <= year <= current_year:
            return year
        
        return np.nan  # Drop anything else
    except:
        return np.nan
merged_train_df['prop.year_built_parsed'] = merged_train_df['prop.year_built'].apply(clean_year_built)
merged_test_df['prop.year_built_parsed'] = merged_test_df['prop.year_built'].apply(clean_year_built)

In [11]:
def parse_num_baths(value):
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value.isdigit():
        return float(value)
    full = 0
    half = 0
    if ':' in value:
        parts = value.split(':') 
        full = float(parts[0])
        half = float(parts[1]) if len(parts) > 1 else 0
    elif 'f' in value or 'h' in value:
        full_match = re.search(r'(\d+)f', value)
        if full_match:
            full = float(full_match.group(1))
        half_match = re.search(r'(\d+)h', value)
        if half_match:
            half = float(half_match.group(1))
    elif 'full' in value or 'half' in value:
        full_match = re.search(r'(\d+)full', value)
        if full_match:
            full = float(full_match.group(1))
        half_match = re.search(r'(\d+)half', value)
        if half_match:
            half = float(half_match.group(1))
    return full + (0.5 * half)
    
merged_train_df['subject.num_baths_parsed'] = merged_train_df['subject.num_baths'].apply(parse_num_baths)
merged_test_df['subject.num_baths_parsed'] = merged_test_df['subject.num_baths'].apply(parse_num_baths)


In [12]:
def combine_bathrooms(df, full_col='full_baths', half_col='half_baths', new_col='num_baths'):
    df = df.copy()
    
    full = df[full_col].fillna(0).astype(float)
    half = df[half_col].fillna(0).astype(float)
    
    df[new_col] = full + 0.5 * half
    return df
merged_train_df = combine_bathrooms(merged_train_df, full_col='prop.full_baths', half_col='prop.half_baths', new_col='prop.num_baths')
merged_test_df = combine_bathrooms(merged_test_df, full_col='prop.full_baths', half_col='prop.half_baths', new_col='prop.num_baths')

In [13]:
def parse_room_count(value):
    """
    Parse and standardize room count values such as '3', '3+1', '2+2', nan, etc.
    Returns a float or np.nan.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip()
    if not value:
        return np.nan
    # Handle values like '3+1', '2+2'
    if '+' in value:
        try:
            parts = [float(x) for x in value.split('+') if x.strip().isdigit()]
            return sum(parts)
        except Exception:
            return np.nan
    # Handle simple numeric values
    try:
        return float(value)
    except Exception:
        return np.nan
merged_train_df['subject.num_beds_parsed'] = merged_train_df['subject.num_beds'].apply(parse_room_count)
merged_test_df['subject.num_beds_parsed'] = merged_test_df['subject.num_beds'].apply(parse_room_count)

In [14]:
import re
def parse_lot_size_sf(lot_size):
    if pd.isna(lot_size):
        return np.nan
    lot_size = lot_size.lower().strip()
    if any(x in lot_size.lower() for x in ['n/a','na','nan']):
        return np.nan
    value = lot_size.replace('+/-','')
    match = re.search(r"([\d.]+)", value)
    if not match:
        return np.nan
    number = float(match[0])
    if any(x in value for x in ['acres', 'acre','ac']):
        return number * 43560
    elif any(x in value for x in ['sqm','sq m']):
        return number * 10.7639
    elif any(x in value for x in ['sqft']):
        return number
    else:
        return np.nan  

merged_train_df['subject.lot_size_sf_parsed'] = merged_train_df['subject.lot_size_sf'].apply(parse_lot_size_sf)
merged_train_df['prop.lot_size_sf_parsed'] = merged_train_df['subject.lot_size_sf'].apply(parse_lot_size_sf)
merged_test_df['subject.lot_size_sf_parsed'] = merged_test_df['subject.lot_size_sf'].apply(parse_lot_size_sf)
merged_test_df['prop.lot_size_sf_parsed'] = merged_test_df['subject.lot_size_sf'].apply(parse_lot_size_sf)

In [15]:
columns_to_drop_X = ['comps','properties', 'subject.basement_area', 'subject.condition',
       'subject.construction', 'subject.effective_date','subject.lot_size_sf','subject.year_built',
       'subject.exterior_finish', 'subject.flooring','subject.room_count',
       'subject.foundation_walls', 'subject.fuel_type', 
       'subject.main_lvl_area', 'subject.municipality_district',
       'subject.plumbing_lines', 'subject.remaining_economic_life',
       'subject.room_total', 'subject.second_lvl_area','subject.num_beds',
       'subject.site_dimensions', 'subject.subject_city_province_zip',
       'subject.third_lvl_area', 'subject.units_sq_ft', 'subject.water_heater','subject.num_baths','prop.full_baths',
       'prop.half_baths','prop.lot_size_sf','prop.year_built',
       'subject.windows','prop.property_sub_type', 'subject.roofing', 'subject.gla','prop.bedrooms','prop.room_count']
merged_train_df['subject.roof'] = merged_train_df['subject.roofing']
merged_train_df['prop.num_beds'] = merged_train_df['prop.bedrooms']
X_train = merged_train_df.drop(columns=columns_to_drop_X)
X_test = merged_test_df.drop(columns=columns_to_drop_X)

In [16]:
X_train.columns

Index(['orderID', 'subject.address', 'subject.structure_type',
       'subject.effective_age', 'subject.style', 'subject.basement',
       'subject.heating', 'subject.cooling', 'subject.subject_age', 'prop.id',
       'prop.address', 'prop.gla', 'prop.city', 'prop.province',
       'prop.postal_code', 'prop.structure_type', 'prop.style', 'prop.levels',
       'prop.roof', 'prop.basement', 'prop.cooling', 'prop.heating',
       'subject.city', 'subject.province', 'subject.postal_code',
       'subject.gla_parsed', 'subject.year_built_parsed',
       'prop.year_built_parsed', 'subject.num_baths_parsed', 'prop.num_baths',
       'subject.num_beds_parsed', 'subject.lot_size_sf_parsed',
       'prop.lot_size_sf_parsed', 'subject.roof', 'prop.num_beds'],
      dtype='object')

In [17]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8129 entries, 0 to 8128
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   orderID                     8129 non-null   int64  
 1   subject.address             8129 non-null   object 
 2   subject.structure_type      8129 non-null   object 
 3   subject.effective_age       8129 non-null   object 
 4   subject.style               8129 non-null   object 
 5   subject.basement            6652 non-null   object 
 6   subject.heating             8129 non-null   object 
 7   subject.cooling             5539 non-null   object 
 8   subject.subject_age         8129 non-null   object 
 9   prop.id                     8129 non-null   int64  
 10  prop.address                8129 non-null   object 
 11  prop.gla                    7978 non-null   float64
 12  prop.city                   8126 non-null   object 
 13  prop.province               8129 

In [18]:
# y_train
comps = train_df[['orderID', 'comps']]
records = []
for _, row in comps.iterrows():
    orderID = row['orderID']
    prop_list = row['comps']
    prop_list = ast.literal_eval(prop_list)
    for prop in prop_list:
    
        prop_record = {
            "orderID": orderID,
            "distance_to_subject": prop.get('distance_to_subject', None),
          "prop_type": prop.get('prop_type', None),
          "stories": prop.get('stories', None),
          "address": prop.get('address', None),
          "city_province": prop.get('city_province', None),
          "sale_date": prop.get('sale_date', None),
          "sale_price": prop.get('sale_price', None),
          "dom": prop.get('dom', None),
          "location_similarity":prop.get('location_similarity', None),
          "lot_size": prop.get('lot_size', None),
          "age":  prop.get('age', None),
          "condition": prop.get('condition', None), 
          "gla": prop.get('gla', None),
          "room_count": prop.get('room_count', None),
          "bed_count":  prop.get('bed_count', None),
          "bath_count": prop.get('bath_count', None),
          "basement_finish": prop.get('basement_finish', None),
          "parking": prop.get('parking', None),
          "neighborhood": prop.get('neighborhood', None)
        }
        records.append(prop_record)

comps_train_df = pd.DataFrame(records)

In [19]:
# y_test
comps = test_df[['orderID', 'comps']]
records = []
for _, row in comps.iterrows():
    orderID = row['orderID']
    prop_list = row['comps']
    prop_list = ast.literal_eval(prop_list)
   
    for prop in prop_list:
    
        prop_record = {
            "orderID": orderID,
            "distance_to_subject": prop.get('distance_to_subject', None),
          "prop_type": prop.get('prop_type', None),
          "stories": prop.get('stories', None),
          "address": prop.get('address', None),
          "city_province": prop.get('city_province', None),
          "sale_date": prop.get('sale_date', None),
          "sale_price": prop.get('sale_price', None),
          "dom": prop.get('dom', None),
          "location_similarity":prop.get('location_similarity', None),
          "lot_size": prop.get('lot_size', None),
          "age":  prop.get('age', None),
          "condition": prop.get('condition', None), 
          "gla": prop.get('gla', None),
          "room_count": prop.get('room_count', None),
          "bed_count":  prop.get('bed_count', None),
          "bath_count": prop.get('bath_count', None),
          "basement_finish": prop.get('basement_finish', None),
          "parking": prop.get('parking', None),
          "neighborhood": prop.get('neighborhood', None)
        }
        records.append(prop_record)
comps_test_df = pd.DataFrame(records)

In [20]:
comps_train_df.columns

Index(['orderID', 'distance_to_subject', 'prop_type', 'stories', 'address',
       'city_province', 'sale_date', 'sale_price', 'dom',
       'location_similarity', 'lot_size', 'age', 'condition', 'gla',
       'room_count', 'bed_count', 'bath_count', 'basement_finish', 'parking',
       'neighborhood'],
      dtype='object')

In [21]:
comps_train_df['num_beds_parsed'] = comps_train_df['bed_count'].apply(parse_room_count)
comps_test_df['num_beds_parsed'] = comps_test_df['bed_count'].apply(parse_room_count)

In [22]:
comps_train_df['num_baths_parsed'] = comps_train_df['bath_count'].apply(parse_num_baths)
comps_test_df['num_baths_parsed'] = comps_test_df['bath_count'].apply(parse_num_baths)

In [23]:

comps_train_df['gla_parsed'] = comps_train_df['gla'].apply(parse_gla)
comps_test_df['gla_parsed'] = comps_test_df['gla'].apply(parse_gla)

In [24]:
def parse_age(value):
    if pd.isna(value):
        return np.nan
    
    value = str(value).strip().lower()
    if value == "new":
        return 0
    if value.isdigit():
        age = float(value)
        if 1900 <= age <= 2025:
            return 2025-age
        return age
    if value.replace('.','',1).isdigit():
        return float(value)
    match = re.search(r"([\d.]+)", value)
    if not match:
        return np.nan
    number = float(match[0]) 
    if 1900 <= number <= 2025:
        return 2025-number
    return number

comps_train_df['age_parsed'] = comps_train_df['age'].apply(parse_age)
comps_test_df['age_parsed'] = comps_test_df['age'].apply(parse_age)

In [25]:
def parse_site_dimensions(dimension):
    if pd.isna(dimension):
        return np.nan
    dimension = str(dimension).strip().lower()
    if any(x in dimension for x in ['condo','common property', 'see','n/a','unknown','not applicable']):
        return np.nan
    try:
        if dimension.replace('.','',1).isdigit():
            return float(dimension)
        if 'acre' in dimension:
            acres = float(dimension.split()[0])
            return acres * 43560
        
        dims = dimension.replace("'",'').replace('"','').replace('ft','').replace('f','')
        dims = dims.replace('x',' ').replace('X',' ').replace('by',' ')
        dims = dims.replace('m','').replace('irregular','').replace('irr','')
        dims = dims.strip().split()
        if len(dims) >= 2:

            length = float(dims[0])
            width = float(dims[1])
            if any(x in dimension.lower() for x in ['m','metre', 'meter']):
                length = length * 3.28084
                width = width * 3.28084
            return length * width
    except:
        return np.nan

    return
comps_train_df['lot_size_parsed'] = comps_train_df['lot_size'].apply(parse_site_dimensions)
comps_test_df['lot_size_parsed'] = comps_test_df['lot_size'].apply(parse_site_dimensions)

In [26]:
columns_to_drop_y = ['distance_to_subject','sale_date','sale_price','dom','location_similarity','condition','parking','neighborhood','lot_size','age','bed_count','bath_count', 'gla','room_count']
y_train = comps_train_df.drop(columns=columns_to_drop_y)
y_test = comps_test_df.drop(columns=columns_to_drop_y)

### Cosine Similarity

In [27]:
X_train.columns

Index(['orderID', 'subject.address', 'subject.structure_type',
       'subject.effective_age', 'subject.style', 'subject.basement',
       'subject.heating', 'subject.cooling', 'subject.subject_age', 'prop.id',
       'prop.address', 'prop.gla', 'prop.city', 'prop.province',
       'prop.postal_code', 'prop.structure_type', 'prop.style', 'prop.levels',
       'prop.roof', 'prop.basement', 'prop.cooling', 'prop.heating',
       'subject.city', 'subject.province', 'subject.postal_code',
       'subject.gla_parsed', 'subject.year_built_parsed',
       'prop.year_built_parsed', 'subject.num_baths_parsed', 'prop.num_baths',
       'subject.num_beds_parsed', 'subject.lot_size_sf_parsed',
       'prop.lot_size_sf_parsed', 'subject.roof', 'prop.num_beds'],
      dtype='object')

In [28]:
y_train.columns

Index(['orderID', 'prop_type', 'stories', 'address', 'city_province',
       'basement_finish', 'num_beds_parsed', 'num_baths_parsed', 'gla_parsed',
       'age_parsed', 'lot_size_parsed'],
      dtype='object')

In [29]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity

# Example dataframe
df = X_train.copy()  # replace with your actual dataframe


In [30]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# List of numerical columns only
numerical_features = [
    'subject.year_built_parsed', 'prop.year_built_parsed',
    'subject.num_baths_parsed', 'prop.num_baths',
    'subject.num_beds_parsed', 'prop.num_beds',
    'subject.lot_size_sf_parsed', 'prop.lot_size_sf_parsed',
    'subject.gla_parsed', 'prop.gla'
]

# Subset your dataframe
X = df[numerical_features]
# display(X.info())

# Separate into subject and prop features
subject_features = X[[col for col in numerical_features if col.startswith('subject.')]]
prop_features = X[[col for col in numerical_features if col.startswith('prop.')]]

# # Impute missing values
imputer = SimpleImputer(strategy='mean')
subject_imputed = imputer.fit_transform(subject_features)
prop_imputed = imputer.fit_transform(prop_features)  # Use fit_transform here if train/test split not used

# # Scale the features
scaler = StandardScaler()
subject_scaled = scaler.fit_transform(subject_imputed)
prop_scaled = scaler.fit_transform(prop_imputed)  # Use fit_transform here too for now

# # Compute cosine similarity
similarity_matrix = cosine_similarity(subject_scaled, prop_scaled)

# Create DataFrame with orderID and top-3 prop.id for each subject
# import numpy as np

top_k = 3
results = []
for i, order_id in enumerate(df['orderID'].unique()):
    row_sim = similarity_matrix[i]
    top_indices = np.argsort(row_sim)[-top_k:][::-1]
    top_prop_ids = df.iloc[top_indices]['prop.id'].tolist()
    results.append({
        'orderID': order_id,
        'top_comps': top_prop_ids
    })

results_df = pd.DataFrame(results)
print(results_df.head())


   orderID               top_comps
0  4768126  [92565, 80203, 287677]
1  4777578  [92565, 80203, 287677]
2  4761220  [92565, 80203, 287677]
3  4758324  [92565, 80203, 287677]
4  4759562  [92565, 80203, 287677]


In [31]:
true_comp_features = ['gla_parsed', 'room_count', 'num_beds_parsed', 
                      'num_baths_parsed', 'age_parsed', 'lot_size_parsed']


In [32]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   orderID           210 non-null    int64  
 1   prop_type         210 non-null    object 
 2   stories           210 non-null    object 
 3   address           210 non-null    object 
 4   city_province     210 non-null    object 
 5   basement_finish   210 non-null    object 
 6   num_beds_parsed   210 non-null    float64
 7   num_baths_parsed  210 non-null    float64
 8   gla_parsed        210 non-null    float64
 9   age_parsed        210 non-null    float64
 10  lot_size_parsed   62 non-null     float64
dtypes: float64(5), int64(1), object(5)
memory usage: 18.2+ KB


In [33]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# Group the true comps by orderID
true_comps_by_order = y_train.groupby('orderID')
order_ids = X_train['orderID'].values  # or df['orderID'].values
# Map from orderID to indices in prediction set
order_to_indices = defaultdict(list)
for idx, oid in enumerate(order_ids):
    order_to_indices[oid].append(idx)

# Feature columns to compare
feature_cols = ['gla_parsed', 'num_beds_parsed', 
                'num_baths_parsed', 'age_parsed', 'lot_size_parsed']

# Normalize true comp features
true_comp_vectors = {}
for oid, group in true_comps_by_order:
    comps = group[feature_cols].dropna()
    if not comps.empty:
        # Store as a matrix of shape (num_comps, num_features)
        true_comp_vectors[oid] = comps.to_numpy(dtype=np.float32)

# Map from (orderID, propID) -> vector
prop_vecs_by_order = defaultdict(list)
for idx, oid in enumerate(order_ids):
    prop_vecs_by_order[oid].append((prop_ids[idx], prop_scaled[idx]))

# Now compute similarity
results = []

for oid in true_comp_vectors:
    if oid not in prop_vecs_by_order:
        continue

    pred_vectors = [vec for _, vec in prop_vecs_by_order[oid]]
    pred_ids = [pid for pid, _ in prop_vecs_by_order[oid]]

    pred_matrix = np.vstack(pred_vectors)
    true_matrix = true_comp_vectors[oid]

    # For each predicted comp, find the max similarity to any true comp
    sim_matrix = cosine_similarity(pred_matrix, true_matrix)  # shape: [#pred, #true]

    top_k = min(3, sim_matrix.shape[0])
    top_k_avg = []

    for i in range(top_k):
        sim_row = sim_matrix[i]  # similarities of this predicted comp to all true comps
        top_sim = np.max(sim_row)
        top_k_avg.append(top_sim)

    avg_similarity = np.mean(top_k_avg)
    results.append((oid, avg_similarity))

# Create results DataFrame
similarity_df = pd.DataFrame(results, columns=['orderID', 'avg_similarity'])
print(f"Mean cosine similarity across all orders: {similarity_df['avg_similarity'].mean():.4f}")


NameError: name 'prop_ids' is not defined

In [ ]:
y_train.columns

Index(['orderID', 'prop_type', 'stories', 'address', 'city_province', 'gla',
       'room_count', 'basement_finish', 'num_beds_parsed', 'num_baths_parsed',
       'gla_parsed', 'age_parsed', 'lot_size_parsed'],
      dtype='object')

In [ ]:
import pandas as pd

# Extract orderID array
order_ids = df['orderID'].values

# Map top 3 prop indices to prop ids (already done)
top_3_prop_ids = prop_ids[top_3_prop_indices]

# Create a DataFrame with orderID and top 3 comps prop IDs
results_df = pd.DataFrame({
    'orderID': order_ids,
    'comp_1_prop_id': top_3_prop_ids[:, 0],
    'comp_2_prop_id': top_3_prop_ids[:, 1],
    'comp_3_prop_id': top_3_prop_ids[:, 2],
})

print(results_df.head())


### Data Pre-Processing

In [ ]:
import ast

properties = df[['orderID', 'properties']]

records = []
for _, row in properties.iterrows():
    orderID = row['orderID']
    prop_list = ddd['properties']
    prop_list = ast.literal_eval(prop_list)
    # print(prop_list)
    # print(type(prop_list))


    if isinstance(prop_list, str):
        try:
            prop_list = json.loads(prop_list.replace("'", '"'))
        except Exception as e:
            # print(f"Skipping row with orderID {orderID} due to JSON error: {e}")
            continue

    if not isinstance(prop_list, list):
        continue
    # print("here")
    for prop in prop_list:
    
        prop_record = {
            "orderID": orderID,
            "id":prop.get("id", None),
            "address": prop.get("address", None),
            "bedrooms": prop.get("bedrooms", None),
            "gla": prop.get("gla", None),
            "city": prop.get("city", None),
            "province": prop.get("province", None),
            "postal_code": prop.get("postal_code", None),
            "property_sub_type": prop.get("property_sub_type", None),
            "structure_type": prop.get("structure_type", None),
            "style": prop.get("style", None),
            "levels": prop.get("levels", None),
            "room_count": prop.get("room_count", None),
            "full_baths": prop.get("full_baths", None),
            "half_baths": prop.get("half_baths", None),
            # "main_level_finished_area": prop.get("main_level_finished_area", None),
            # "upper_lvl_fin_area": prop.get("upper_lvl_fin_area", None),
            # "bg_fin_area": prop.get("bg_fin_area", None),
            "lot_size_sf": prop.get("lot_size_sf", None),
            "year_built": prop.get("year_built", None),
            "roof": prop.get("roof", None),
            "basement": prop.get("basement", None),
            "cooling": prop.get("cooling", None),
            "heating": prop.get("heating", None),
            # "close_price": prop.get("close_price", None),
            # "close_date": prop.get("close_date", None),
            # "public_remarks": prop.get("public_remarks", None),
            # "latitude": prop.get("latitude", None),
            # "longitude": prop.get("longitude", None),
        }
        records.append(prop_record)

properties_df = pd.DataFrame(records)
properties_df = properties_df.rename(columns={
    col: f"prop.{col}" for col in properties_df.columns if col != "orderID"
})
display(properties_df.head())

NameError: name 'ddd' is not defined

In [ ]:
merged_df = df.merge(properties_df, on='orderID', how='left')
merged_df.columns

Index(['orderID', 'comps', 'properties', 'subject.address',
       'subject.subject_city_province_zip', 'subject.effective_date',
       'subject.municipality_district', 'subject.site_dimensions',
       'subject.lot_size_sf', 'subject.units_sq_ft', 'subject.year_built',
       'subject.structure_type', 'subject.roofing', 'subject.effective_age',
       'subject.style', 'subject.construction',
       'subject.remaining_economic_life', 'subject.windows',
       'subject.basement', 'subject.exterior_finish', 'subject.basement_area',
       'subject.foundation_walls', 'subject.flooring',
       'subject.plumbing_lines', 'subject.heating', 'subject.fuel_type',
       'subject.water_heater', 'subject.cooling', 'subject.room_count',
       'subject.num_beds', 'subject.room_total', 'subject.main_lvl_area',
       'subject.second_lvl_area', 'subject.third_lvl_area', 'subject.gla',
       'subject.subject_age', 'subject.num_baths', 'subject.condition',
       'prop.id', 'prop.address', 'prop.be

In [ ]:
columns_to_drop = ['properties', 'subject.basement_area', 'subject.condition',
       'subject.construction', 'subject.effective_date',
       'subject.exterior_finish', 'subject.flooring',
       'subject.foundation_walls', 'subject.fuel_type', 'subject.heating',
       'subject.main_lvl_area', 'subject.municipality_district',
       'subject.plumbing_lines', 'subject.remaining_economic_life',
       'subject.room_total', 'subject.second_lvl_area',
       'subject.site_dimensions', 'subject.subject_city_province_zip',
       'subject.third_lvl_area', 'subject.units_sq_ft', 'subject.water_heater',
       'subject.windows']
data = merged_df.drop(columns=columns_to_drop)
print("Columns dropped. New shape:", data.shape)
display(data.head())
data.columns


Columns dropped. New shape: (9820, 36)


,orderID,comps,subject.address,subject.lot_size_sf,subject.year_built,subject.structure_type,subject.roofing,subject.effective_age,subject.style,subject.basement,...,prop.levels,prop.room_count,prop.full_baths,prop.half_baths,prop.lot_size_sf,prop.year_built,prop.roof,prop.basement,prop.cooling,prop.heating
0,4762597,"[{'distance_to_subject': '0.15 KM', 'prop_type...",142-950 Oakview Ave Kingston ON K7M 6W8,NaN,1976,Townhouse,Asphalt Shingle,25,2 Storey,Full/Finished,...,Two,11.0,3.0,NaN,3555.5,NaN,None,Unfinished,Central Air,Forced Air
1,4762597,"[{'distance_to_subject': '0.15 KM', 'prop_type...",142-950 Oakview Ave Kingston ON K7M 6W8,NaN,1976,Townhouse,Asphalt Shingle,25,2 Storey,Full/Finished,...,2-Storey,11.0,NaN,NaN,3535.0,NaN,,Unfinished,Central Air,Forced Air
2,4762597,"[{'distance_to_subject': '0.15 KM', 'prop_type...",142-950 Oakview Ave Kingston ON K7M 6W8,NaN,1976,Townhouse,Asphalt Shingle,25,2 Storey,Full/Finished,...,Two,11.0,4.0,NaN,2622.0,NaN,None,Fin W/O,Central Air,Forced Air
3,4762597,"[{'distance_to_subject': '0.15 KM', 'prop_type...",142-950 Oakview Ave Kingston ON K7M 6W8,NaN,1976,Townhouse,Asphalt Shingle,25,2 Storey,Full/Finished,...,2-Storey,11.0,NaN,NaN,2622.0,NaN,,Finished with Walk-Out,Central Air,Forced Air
4,4762597,"[{'distance_to_subject': '0.15 KM', 'prop_type...",142-950 Oakview Ave Kingston ON K7M 6W8,NaN,1976,Townhouse,Asphalt Shingle,25,2 Storey,Full/Finished,...,None,13.0,4.0,0.0,16672.0,NaN,None,"Part Bsmt, Unfinished",None,Baseboard


Index(['orderID', 'comps', 'subject.address', 'subject.lot_size_sf',
       'subject.year_built', 'subject.structure_type', 'subject.roofing',
       'subject.effective_age', 'subject.style', 'subject.basement',
       'subject.cooling', 'subject.room_count', 'subject.num_beds',
       'subject.gla', 'subject.subject_age', 'subject.num_baths', 'prop.id',
       'prop.address', 'prop.bedrooms', 'prop.gla', 'prop.city',
       'prop.province', 'prop.postal_code', 'prop.property_sub_type',
       'prop.structure_type', 'prop.style', 'prop.levels', 'prop.room_count',
       'prop.full_baths', 'prop.half_baths', 'prop.lot_size_sf',
       'prop.year_built', 'prop.roof', 'prop.basement', 'prop.cooling',
       'prop.heating'],
      dtype='object')

In [ ]:
import re

import pandas as pd
import re

def standardize_and_split_address(address):
    """
    Splits an address string into address, city, province, and postal_code.
    Returns a pandas Series: (address, city, province, postal_code)
    Handles both comma-separated and space-separated addresses.
    """
    if pd.isna(address):
        return pd.Series([None, None, None, None])
    
    addr = str(address).strip().replace('"', '').replace("'", "")
    
    # Extract postal code using Canadian postal code pattern
    postal_code_pattern = r'([A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d)'
    postal_code_match = re.search(postal_code_pattern, addr)
    postal_code = postal_code_match.group(1) if postal_code_match else None
    
    if postal_code:
        addr_wo_postal = addr.replace(postal_code, '').strip(', ')
    else:
        addr_wo_postal = addr
    
    # Check if address contains commas
    if ',' in addr_wo_postal:
        parts = [p.strip() for p in addr_wo_postal.split(',') if p.strip()]
        
        province = None
        city = None
        address_part = None
        
        if len(parts) >= 2:
            province_match = re.search(r'\b([A-Z]{2})\b', parts[-1])
            if province_match:
                province = province_match.group(1)
                city = parts[-2]
                address_part = ', '.join(parts[:-2])
            else:
                city = parts[-1]
                address_part = ', '.join(parts[:-1])
        elif len(parts) == 1:
            address_part = parts[0]
        else:
            address_part = addr_wo_postal
    else:
        # Space separated, try to parse from tokens
        tokens = addr_wo_postal.split()
        
        province = None
        city = None
        address_part = None
        
        # Assume last token is province if it is 2 uppercase letters
        if len(tokens) >= 2 and re.fullmatch(r'[A-Z]{2}', tokens[-1]):
            province = tokens[-1]
            if province == "ON":
                province = "Ontario"
            elif province == "BC":
                province = "British Columbia"
            elif province == "AB":
                province = "Alberta"
            elif province == "NS": 
                province = "Nova Scotia"
            elif province == "None":
                return np.nan
            city = tokens[-2]
            address_part = " ".join(tokens[:-2])
        elif len(tokens) >= 1:
            # fallback if no province token detected
            address_part = " ".join(tokens[:-1])
            city = tokens[-1]
        else:
            address_part = addr_wo_postal
    
    return pd.Series([address_part, city, province, postal_code])

# Apply to normalized_df
data[['subject.address', 'subject.city', 'subject.province', 'subject.postal_code']] = data['subject.address'].apply(standardize_and_split_address)

In [ ]:
data.columns

Index(['orderID', 'comps', 'subject.address', 'subject.lot_size_sf',
       'subject.year_built', 'subject.structure_type', 'subject.roofing',
       'subject.effective_age', 'subject.style', 'subject.basement',
       'subject.cooling', 'subject.room_count', 'subject.num_beds',
       'subject.gla', 'subject.subject_age', 'subject.num_baths', 'prop.id',
       'prop.address', 'prop.bedrooms', 'prop.gla', 'prop.city',
       'prop.province', 'prop.postal_code', 'prop.property_sub_type',
       'prop.structure_type', 'prop.style', 'prop.levels', 'prop.room_count',
       'prop.full_baths', 'prop.half_baths', 'prop.lot_size_sf',
       'prop.year_built', 'prop.roof', 'prop.basement', 'prop.cooling',
       'prop.heating', 'subject.city', 'subject.province',
       'subject.postal_code'],
      dtype='object')

In [ ]:
def parse_year_built(value):
    if pd.isna(value):
        return np.nan
    
    value = str(value).strip().lower()
    if value == "new":
        return 2025
    if value.isdigit():
        return float(value)
    if value.replace('.','',1).isdigit():
        return float(value)
    match = re.search(r"([\d.]+)", value)
    if not match:
        return np.nan
    number = float(match[0]) 
    return number

data['subject.year_built_parsed'] = data['subject.year_built'].apply(parse_year_built)

print("\n After parsing year_built_parsed:")
print(data['subject.year_built_parsed'].unique())
data['subject.year_built_parsed'].describe()




 After parsing year_built_parsed:
[1976. 2011. 1983. 2012. 1978. 1985. 1941. 2021. 2006. 2019. 2024. 2000.
 1982. 2010. 2002. 1968. 1890. 1993. 2016. 1952. 2018. 2017. 1951. 2008.
 2013. 2004. 2003. 2025. 1972. 1845. 1995. 1990. 1961. 1970. 1910. 1940.
 1989. 1885. 1900. 1974. 2022. 2015. 1998. 1975. 2007. 1950. 1988.   nan
 1971. 1992. 1999. 1979. 1977. 1920.]


count    9788.000000
mean     1994.662648
std        32.966353
min      1845.000000
25%      1979.000000
50%      2004.000000
75%      2017.000000
max      2025.000000
Name: subject.year_built_parsed, dtype: float64

In [ ]:
data['prop.year_built'].unique()

array([      nan, 4.000e+01, 7.500e+01, 1.000e+01, 2.010e+03, 1.967e+03,
       2.021e+03, 1.918e+03, 2.013e+03, 1.995e+03, 1.945e+03, 1.934e+03,
       1.998e+03, 1.980e+03, 1.875e+03, 1.950e+03, 2.004e+03, 1.991e+03,
       2.017e+03, 2.015e+03, 2.014e+03, 2.012e+03, 2.018e+03, 2.023e+03,
       2.016e+03, 2.006e+03, 2.003e+03, 2.020e+03, 2.022e+03, 1.992e+03,
       2.009e+03, 1.983e+03, 2.007e+03, 1.870e+03, 2.011e+03, 1.985e+03,
       2.002e+03, 2.019e+03, 1.981e+03, 1.900e+03, 2.024e+03, 1.984e+03,
       1.954e+03, 1.969e+03, 1.948e+03, 1.978e+03, 2.000e+03, 1.988e+03,
       1.971e+03, 1.975e+03, 1.973e+03, 1.972e+03, 1.940e+03, 1.994e+03,
       1.982e+03, 1.965e+03, 1.952e+03, 1.996e+03, 1.989e+03, 2.001e+03,
       1.935e+03, 1.963e+03, 1.977e+03, 1.986e+03, 1.979e+03, 1.970e+03,
       1.974e+03, 1.999e+03, 1.993e+03, 1.987e+03, 1.976e+03, 2.008e+03,
       2.300e+01, 1.956e+03, 1.958e+03, 1.944e+03, 1.938e+03, 1.990e+03,
       1.943e+03, 1.946e+03, 1.955e+03, 1.953e+03, 

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

def clean_year_built(value):
    try:
        # Convert from scientific notation or string to float
        year = float(value)
        year = int(round(year))  # Convert to integer
        
        current_year = datetime.now().year
        
        # Handle likely invalid placeholder years (e.g., 40, 8, 13)
        if year < 100:  
            return np.nan

        # Handle likely valid years (e.g., 1900–current year)
        if 1800 <= year <= current_year:
            return year
        
        return np.nan  # Drop anything else
    except:
        return np.nan
data['prop.year_built_parsed'] = data['prop.year_built'].apply(clean_year_built)
print("\n After cleaning year_built_cleaned:") 
print(data['prop.year_built_parsed'].unique())



 After cleaning year_built_cleaned:
[  nan 2010. 1967. 2021. 1918. 2013. 1995. 1945. 1934. 1998. 1980. 1875.
 1950. 2004. 1991. 2017. 2015. 2014. 2012. 2018. 2023. 2016. 2006. 2003.
 2020. 2022. 1992. 2009. 1983. 2007. 1870. 2011. 1985. 2002. 2019. 1981.
 1900. 2024. 1984. 1954. 1969. 1948. 1978. 2000. 1988. 1971. 1975. 1973.
 1972. 1940. 1994. 1982. 1965. 1952. 1996. 1989. 2001. 1935. 1963. 1977.
 1986. 1979. 1970. 1974. 1999. 1993. 1987. 1976. 2008. 1956. 1958. 1944.
 1938. 1990. 1943. 1946. 1955. 1953. 1951. 1997. 1939. 1949. 2025. 1915.
 1947. 1926. 1959. 2005. 1936. 1968. 1961. 1850. 1964. 1962. 1957. 1966.
 1912. 1911. 1914. 1910. 1929. 1913. 1922. 1928. 1937. 1925. 1941. 1907.
 1917. 1908. 1923. 1905. 1901. 1932. 1960. 1942. 1863. 1829. 1920. 1931.
 1924. 1930. 1903. 1919. 1933. 1885.]


In [ ]:
display(data['prop.half_baths'].unique())

array([nan,  0.,  1.,  2.,  3.])

In [ ]:
def parse_num_baths(value):
    if pd.isna(value):
        return np.nan
    value = str(value).strip().lower()
    if value.isdigit():
        return float(value)
    full = 0
    half = 0
    if ':' in value:
        parts = value.split(':') 
        full = float(parts[0])
        half = float(parts[1]) if len(parts) > 1 else 0
    elif 'f' in value or 'h' in value:
        full_match = re.search(r'(\d+)f', value)
        if full_match:
            full = float(full_match.group(1))
        half_match = re.search(r'(\d+)h', value)
        if half_match:
            half = float(half_match.group(1))
    elif 'full' in value or 'half' in value:
        full_match = re.search(r'(\d+)full', value)
        if full_match:
            full = float(full_match.group(1))
        half_match = re.search(r'(\d+)half', value)
        if half_match:
            half = float(half_match.group(1))
    return full + (0.5 * half)
    
data['subject.num_baths_parsed'] = data['subject.num_baths'].apply(parse_num_baths)


print("\n Values that counlent be parsed")
print(data[data['subject.num_baths_parsed'].isnull()]['subject.num_baths'].unique())


 Values that counlent be parsed
[nan]


In [ ]:
def combine_bathrooms(df, full_col='full_baths', half_col='half_baths', new_col='num_baths'):
    df = df.copy()
    
    # Replace None/NaN with 0, convert to float, and compute
    full = df[full_col].fillna(0).astype(float)
    half = df[half_col].fillna(0).astype(float)
    
    df[new_col] = full + 0.5 * half
    return df
data = combine_bathrooms(data, full_col='prop.full_baths', half_col='prop.half_baths', new_col='prop.num_baths')

In [ ]:
data['subject.gla'].unique()

array([1044.,   nan])

In [ ]:
data['subject.gla_parsed'] = pd.to_numeric(df['subject.gla'], errors='coerce')

In [ ]:
data['subject.num_baths_parsed'].unique()

array([1.5, 2.5, 3. , 2. , 3.5, 1. , 0.5, 4. , nan])

In [ ]:
import numpy as np

def parse_room_count(value):
    """
    Parse and standardize room count values such as '3', '3+1', '2+2', nan, etc.
    Returns a float or np.nan.
    """
    if pd.isna(value):
        return np.nan
    value = str(value).strip()
    if not value:
        return np.nan
    # Handle values like '3+1', '2+2'
    if '+' in value:
        try:
            parts = [float(x) for x in value.split('+') if x.strip().isdigit()]
            return sum(parts)
        except Exception:
            return np.nan
    # Handle simple numeric values
    try:
        return float(value)
    except Exception:
        return np.nan

# Example usage:
data['subject.num_beds_parsed'] = data['subject.num_beds'].apply(parse_room_count)

In [ ]:
import re
def parse_lot_size_sf(lot_size):
    if pd.isna(lot_size):
        return np.nan
    lot_size = lot_size.lower().strip()
    if any(x in lot_size.lower() for x in ['n/a','na','nan']):
        return np.nan
    value = lot_size.replace('+/-','')
    match = re.search(r"([\d.]+)", value)
    if not match:
        return np.nan
    number = float(match[0])
    if any(x in value for x in ['acres', 'acre','ac']):
        return number * 43560
    elif any(x in value for x in ['sqm','sq m']):
        return number * 10.7639
    elif any(x in value for x in ['sqft']):
        return number
    else:
        return np.nan  

data['subject.lot_size_sf_parsed'] = data['subject.lot_size_sf'].apply(parse_lot_size_sf)
data['prop.lot_size_sf_parsed'] = data['subject.lot_size_sf'].apply(parse_lot_size_sf)

In [ ]:
columns = ['subject.lot_size_sf_parsed','prop.lot_size_sf_parsed','subject.city', 'subject.province','subject.num_baths_parsed',
       'subject.postal_code','subject.address','prop.address','prop.city','subject.gla_parsed','subject.num_beds_parsed',
       'prop.province', 'prop.postal_code','subject.year_built_parsed','prop.year_built_parsed', 'subject.room_count','prop.room_count',
       'prop.gla','prop.bedrooms','prop.num_baths','prop.id','orderID']

In [ ]:
X_train = train_df[[col for col in columns if col in train_df.columns]].copy()
y_train = train_df[['orderID', 'comps']].copy()

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (70, 3)
y_train shape: (70, 2)


In [ ]:
import ast
comps = y_train.copy()
records = []
for _, row in comps.iterrows():
    orderID = row['orderID']
    prop_list = row['comps']
    prop_list = ast.literal_eval(prop_list)
    if isinstance(prop_list, str):
        try:
            prop_list = json.loads(prop_list.replace("'", '"'))
        except Exception as e:
            # print(f"Skipping row with orderID {orderID} due to JSON error: {e}")
            continue

    if not isinstance(prop_list, list):
        continue
    for prop in prop_list:
    
        prop_record = {
            "orderID": orderID,
            "distance_to_subject": prop.get('distance_to_subject', None),
          "prop_type": prop.get('prop_type', None),
          "stories": prop.get('stories', None),
          "address": prop.get('address', None),
          "city_province": prop.get('city_province', None),
          "sale_date": prop.get('sale_date', None),
          "sale_price": prop.get('sale_price', None),
          "dom": prop.get('dom', None),
          "location_similarity":prop.get('location_similarity', None),
          "lot_size": prop.get('lot_size', None),
          "age":  prop.get('age', None),
          "condition": prop.get('condition', None), 
          "gla": prop.get('gla', None),
          "room_count": prop.get('room_count', None),
          "bed_count":  prop.get('bed_count', None),
          "bath_count": prop.get('bath_count', None),
          "basement_finish": prop.get('basement_finish', None),
          "parking": prop.get('parking', None),
          "neighborhood": prop.get('neighborhood', None)
        }
        records.append(prop_record)

comps_df = pd.DataFrame(records)
display(comps_df.head())

,orderID,distance_to_subject,prop_type,stories,address,city_province,sale_date,sale_price,dom,location_similarity,lot_size,age,condition,gla,room_count,bed_count,bath_count,basement_finish,parking,neighborhood
0,4768126,0.24 KM,Townhouse,2 Storey,550 Paine Ave,Kanata ON K2T 0K7,Feb/21/2025,"650,000",7 +/-,Similar,25x 98 / 3035.35 sf +/-,2016 +/-,Similar,1570 SqFt,6,3,2:1,Full/Finished,Sgl. Att. Gar.,
1,4768126,0.33 KM,Townhouse,2 Storey,524 Paine Ave,Kanata ON K2T 0K7,Feb/27/2025,"676,000",27 +/-,Similar,27 x 98 / 2664.87 sf +/-,2015 +/-,Similar,1869 SqFt,8,5,2:1,Full/Finished,Sgl. Att. Gar.,
2,4768126,0.11 KM,Townhouse,2 Storey,164 Calvington Ave,Kanata ON K2T 0H8,Mar/26/2025,"706,000",2 +/-,Similar,22 x 89 / 3159.74 sf +/-,2013 +/-,Similar,1830 SqFt,7,4,2:1,Full/Finished,Sgl. Att. Gar.,
3,4777578,0.68 KM,Detached,2 Storey,565 Boulder Creek Cir,Langdon AB T0J 1X3,Mar/28/2025,"826,000",42,Similar,7840 SqFt,17,Inferior,2342 SqFt,8,3,2:1,Unfinished/Walkout,Quad or More Attached,
4,4777578,0.38 KM,Detached,2 Storey,264 Boulder Creek Dr,Langdon AB T0J 1X3,Apr/25/2025,"930,000",18,Inferior,8276 SqFt,11,Similar,2700 SqFt,8,3,3:1,Full/Finished,Quad or More Attached,


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
!pip install xgboost
import xgboost as xgb
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = train_df[columns].copy()

df = df.drop(columns=['subject.address', 'prop.address', 'subject.postal_code', 'prop.postal_code'])

# Identify numeric and categorical columns
num_cols = [col for col in df.columns if any(key in col for key in ['lot_size', 'gla', 'num_baths', 'room_count', 'year_built', 'num_beds', 'bedrooms'])]
cat_cols = list(set(df.columns) - set(num_cols) - {'order.id', 'prop.id'})

# Create preprocessing pipeline for numeric and categorical separately
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), num_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_cols)
    ]
)
X_preprocessed = preprocessor.fit_transform(df)


KeyError: "['subject.lot_size_sf_parsed', 'prop.lot_size_sf_parsed', 'subject.city', 'subject.province', 'subject.num_baths_parsed', 'subject.postal_code', 'prop.address', 'prop.city', 'subject.gla_parsed', 'subject.num_beds_parsed', 'prop.province', 'prop.postal_code', 'subject.year_built_parsed', 'prop.year_built_parsed', 'prop.room_count', 'prop.gla', 'prop.bedrooms', 'prop.num_baths', 'prop.id'] not in index"

In [ ]:
!pip install kneed


In [ ]:
subject_cols = [col for col in df.columns if col.startswith('subject.')]
prop_cols = [col for col in df.columns if col.startswith('prop.') and col not in ['prop.id']]

# Get corresponding indices
subject_indices = [i for i, col in enumerate(df.columns) if col in subject_cols]
prop_indices = [i for i, col in enumerate(df.columns) if col in prop_cols]

# Slice preprocessed features into subject and prop parts
subject_feats = X_preprocessed[:, subject_indices]
prop_feats = X_preprocessed[:, prop_indices]

In [ ]:
similarity = cosine_similarity(subject_feats, prop_feats)

# Get top 3 most similar comp indices for each subject
top_3_indices = np.argsort(similarity, axis=1)[:, -3:][:, ::-1]  # top 3, in descending order

In [ ]:
prop_ids = df['prop.id'].values
top_3_prop_ids = np.array([[prop_ids[j] for j in row] for row in top_3_indices])

# Build final result dataframe
results_df = df[['orderID']].copy()
results_df['comp_1_prop_id'] = top_3_prop_ids[:, 0]
results_df['comp_2_prop_id'] = top_3_prop_ids[:, 1]
results_df['comp_3_prop_id'] = top_3_prop_ids[:, 2]

results_df = results_df.drop_duplicates(subset=['orderID'])

# Show result
display(results_df.head())